# Contorl of PRR (UnderActued setting): applicability of Input Decouplig through Collocated Form

###    Abstract 
Project for the class of Underactuated robotics of professor L. Lanari.

Regulation task for a planar robot through Cartesian force applied at the end-effector level. \\
After prove that the definition of Collocated Form suit to this kind of structure, we will perform the regulation task and obtain a partition of the actuation matrix A(q) like :
$$ A(q) = 
\begin{bmatrix}
     I &\\
     0 &
\end{bmatrix}
$$


The kind of control that can be achive with this procedure could be easily apply to medical tasks, soft robots and actuator fault tolerant system in cooperative tasks. 

### On the Integrability condition for the collacated form

Defining the passive output as:
$$ \dot y = A^{T}(q)\dot q $$
end considering the dynamic model of such robot as
$$
\begin{equation}
 M(q)\ddot q + c(q, \dot q) = J^T F
\end{equation}
$$
we can easily reconize that the $A^T$ matrix is nothing else that the Jacobian of the system, and so we can conclude that out passive output $\dot y$ is the velocity of the end effector.


Thanks to this result, we can claim that any system with similar dynamics, can be considered Collocated, AKA admit a solution to the Input Decoupling problem through a change of cordinate

In [1]:
"""## importing"""
import numpy as np
import sys


from pydrake.common.value import Value
from pydrake.geometry import StartMeshcat, MeshcatVisualizer
from pydrake.systems.framework import DiagramBuilder, LeafSystem, BasicVector
from pydrake.multibody.math import SpatialForce
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph, ExternallyAppliedSpatialForce, ExternallyAppliedSpatialForceMultiplexer
from pydrake.multibody.parsing import Parser
from pydrake.all import Simulator


# pydot = interface to graphViz,consist in a parser for the DotFormat used by GraphViz
from pydot import graph_from_dot_data


In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [3]:
class Controller(LeafSystem):

    def __init__(self): 
        LeafSystem.__init__(self)
        self.DeclareVectorOutputPort("y", BasicVector(2), self.CalcOutputY)
        self.t = 50
    def CalcOutputY(self, context, output): 
        q1dotdot = 0
        q2dotdot = 0
        output.SetFromVector([q1dotdot, q2dotdot])
    
class ForceControllerPD(LeafSystem):
    
    def __init__(self, plant, Kp=10, Kd=100):
        LeafSystem.__init__(self)
        self.DeclareVectorInputPort("state", plant.num_multibody_states())
        self.DeclareVectorInputPort("target_position", BasicVector(2))
        self.DeclareVectorOutputPort("cartesianForce", BasicVector(2), self.CalcForceControl)

        self.Kp = Kp
        self.Kd = Kd
        self.link_len = 0.5

    def CalcForceControl(self, context, F):
        state = self.get_input_port(0).Eval(context)
        q = state[0], state[1]
        q_dot = state[8], state[9]
        x, y, x_dot, y_dot = self.kinematic_and_diffKin(q, q_dot)
        x_req, y_req = self.get_input_port(1).Eval(context)
        fx = self.Kp*(x_req - x) - self.Kp*(x_dot)
        fy = self.Kp*(y_req - y) - self.Kp*(y_dot)
        F.SetFromVector([fx, fy])

    def kinematic_and_diffKin(self, q, q_dot):

        x = self.link_len*(np.cos(q[0]) + np.cos([q[0] + q[1]]))
        y = self.link_len*(np.sin(q[0]) + np.sin([q[0] + q[1]]))

        x_dot = (-self.link_len*(np.sin(q[0]) + np.sin(q[0] + q[1]))*q_dot[0]) -(self.link_len*np.sin(q[0]+q[1]))*q_dot[1]
        y_dot = self.link_len*(np.cos(q[0]) + np.cos(q[0] + q[1]))*q_dot[0] + self.link_len*np.cos(q[0] + q[1])*q_dot[1]

        return x, y, x_dot, y_dot




class EndEffector_force_input_Sys(LeafSystem):
    def __init__(self, end_effector_id):
        LeafSystem.__init__(self)
        ## input
        self.DeclareVectorInputPort("cartesian Force", BasicVector(2))

        ##output
        self.force_obj = ExternallyAppliedSpatialForce()
        self.force_obj.body_index = end_effector_id
        force_cls = Value[list[ExternallyAppliedSpatialForce]]
        self.DeclareAbstractOutputPort("F-cartesian", 
                                       lambda: force_cls(), 
                                       self.CalcSpatialForce)

    def CalcSpatialForce(self, context, applied_force):
        fx, fy = self.get_input_port(0).Eval(context)
        print(fx, fy)
        F = SpatialForce(tau=np.zeros(3), f=np.array([fx, fy, 0]))
        self.force_obj.F_Bq_W = F
        applied_force.set_value([self.force_obj])



# building diagram, connecting port


In [4]:
def build_system():

    meshcat.Delete()
    meshcat.DeleteAddedControls()

    builder = DiagramBuilder()
    
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
    Parser(plant, scene_graph).AddModels('../sdf_model/RR_planar.sdf')
    plant.Finalize()

    controller = builder.AddSystem(ForceControllerPD(plant))
    
    apply_force_sys = builder.AddSystem(EndEffector_force_input_Sys(plant.GetBodyByName('end-effector').index()))

    builder.Connect(apply_force_sys.get_output_port(0), plant.get_applied_spatial_force_input_port())
    builder.Connect(controller.get_output_port(), apply_force_sys.get_input_port())    
    builder.Connect(plant.get_state_output_port(), controller.get_input_port(0))

    visualizer = MeshcatVisualizer.AddToBuilder(
    builder, 
    scene_graph.get_query_output_port(),
    meshcat
    )

    diagram = builder.Build()
    
    root_context = diagram.CreateDefaultContext()
    plant_context = plant.GetMyContextFromRoot(root_context)
    controller_context = controller.GetMyContextFromRoot(root_context)

    # set target position 
    controller.get_input_port(1).FixValue(controller_context, np.array([-1, -10]))
    
    # torque to 0 : simulate passivity or actuator fault
    plant.get_actuation_input_port().FixValue(plant_context, np.zeros(2))


    

    return diagram, visualizer, plant, root_context


diagram, plant, visual, root_context = build_system()
simulator = Simulator(diagram, root_context)
simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(1.5)


-20.0 -100.0
-20.0 -99.999999019
-20.0 -99.999997057
-20.0 -99.999994114
-20.0 -99.99999019
-20.0 -99.999985285
-20.0 -99.999979399
-20.0 -99.999972532
-20.0 -99.999964684
-20.0 -99.999955855
-20.0 -99.999946045
-20.0 -99.999935254
-20.0 -99.999923482
-20.0 -99.999910729
-20.0 -99.999896995
-20.0 -99.99988228
-20.0 -99.999866584
-20.0 -99.999849907
-20.0 -99.999832249
-20.0 -99.99981361
-20.0 -99.99979399
-20.0 -99.999773389
-20.0 -99.999751807
-20.0 -99.999729244
-20.0 -99.9997057
-20.0 -99.999681175
-20.0 -99.999655669
-20.0 -99.999629182
-20.0 -99.999601714
-20.0 -99.999573265
-20.0 -99.999543835
-20.0 -99.999513424
-20.0 -99.999482032
-20.0 -99.999449659
-20.0 -99.999416305
-20.0 -99.99938197
-20.0 -99.999346654
-20.0 -99.999310357
-20.0 -99.999273079
-20.0 -99.99923482
-20.0 -99.99919558
-20.0 -99.999155359
-20.0 -99.999114157
-20.0 -99.999071974
-20.0 -99.99902881
-20.0 -99.998984665
-20.0 -99.998939539
-20.0 -99.998893432
-20.0 -99.998846344
-20.0 -99.998798275
-20.0 -99.9987492

In [ ]:
## controller
end_effector_id = plant.GetBodyByName('end-effector').index()
CartesianController_sys = EndEffector_force_input_Sys(end_effector_id)
builder.AddSystem(CartesianController_sys)
builder.Connect(CartesianController_sys.get_output_port(0), plant.get_applied_spatial_force_input_port())

In [60]:
builder = DiagramBuilder()
    
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
Parser(plant, scene_graph).AddModels('../sdf_model/RR_planar.sdf')
plant.Finalize()

diagram  = builder.Build()
root_context = diagram.CreateDefaultContext()

plant_context = plant.GetMyContextFromRoot(root_context)
plant.GetJointByName('q1').set_angle(plant_context, -np.pi)
plant.GetJointByName('q2').set_angle(plant_context, np.pi/2)

state_port = plant.get_state_output_port()



In [72]:
plant.num_positions(), plant.num_velocities(), plant.num_multibody_states()

(9, 8, 17)

In [ ]:
diagram = builder.Build()
diagram_root_context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(diagram_root_context)
Controller_context = CartesianController_sys.GetMyContextFromRoot(diagram_root_context)    

In [ ]:
## set initial position
plant.GetJointByName('q1').set_angle(plant_context, -np.pi)
plant.GetJointByName('q2').set_angle(plant_context, np.pi/2)
diagram.ForcedPublish(diagram_root_context)

## simulate actuation fault
plant.get_actuation_input_port().FixValue(plant_context, np.zeros(2))


# Simulate and save data

In [ ]:
simulator = Simulator(diagram, diagram_root_context)
simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(5.0 if running_as_notebook else 0.1)

RuntimeError: C++ object must be owned by pybind11 when attempting to release to C++

In [ ]:
def save_plant_svg_graph(diagram, destination_path):
    dot_format_graph = graph_from_dot_data(diagram.GetGraphvizString())[0]
    binary_data_svg = dot_format_graph.create_svg()
    with open(destination_path, 'wb') as f:
        f.write(binary_data_svg)

save_plant_svg_graph(diagram, destination_path="../log/RR_planar.svg")